In [70]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from snowflake.connector.pandas_tools import write_pandas
from sqlalchemy import create_engine
import snowflake.connector

In [71]:
import snowflake.connector

In [72]:
## Set options to view all columns
pd.set_option('display.max_columns', None)

In [73]:
SNOWFLAKE_USER = 'kbharaj3'
SNOWFLAKE_PASSWORD = 'Snowfl@key0014'
SNOWFLAKE_ACCOUNT = 'qx25653.ca-central-1.aws'
SNOWFLAKE_WAREHOUSE = 'FOOTY_STORE'
SNOWFLAKE_DATABASE = 'GEGENSTATS'
SNOWFLAKE_SCHEMA = 'TABLES'

In [74]:
conn = snowflake.connector.connect(
    user=SNOWFLAKE_USER,
    password=SNOWFLAKE_PASSWORD,
    account=SNOWFLAKE_ACCOUNT,
    warehouse=SNOWFLAKE_WAREHOUSE,
    database=SNOWFLAKE_DATABASE,
    schema=SNOWFLAKE_SCHEMA
)

In [75]:
cursor = conn.cursor()

In [99]:
cursor.execute('SELECT * FROM TEAMS')
team_rows = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
team_names = pd.DataFrame(team_rows, columns=column_names)

In [100]:
cursor.execute('SELECT * FROM TEAM_STANDARD_STATS')
standard_rows = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
team_standard = pd.DataFrame(standard_rows, columns=column_names)

In [101]:
cursor.execute('SELECT * FROM TEAM_ATTACKING_STATS')
attacking_rows = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
team_attacking = pd.DataFrame(attacking_rows, columns=column_names)

In [102]:
cursor.execute('SELECT * FROM TEAM_DEFENDING_STATS')
defending_rows = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
team_defending = pd.DataFrame(defending_rows, columns=column_names)

In [103]:
cursor.execute('SELECT * FROM COMPETITIONS')
competition_rows = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
df_competitions = pd.DataFrame(competition_rows, columns=column_names)

In [104]:
team_names.drop(['TEAM_LOGO_URL'], axis=1, inplace=True)

In [105]:
team_standard = team_standard.merge(team_names, on='TEAM_FBREF_ID', how='left')
team_defending = team_defending.merge(team_names, on='TEAM_FBREF_ID', how='left')
team_attacking = team_attacking.merge(team_names, on='TEAM_FBREF_ID', how='left')

In [106]:
team_attacking = team_attacking.merge(team_standard, on=['TEAM_FBREF_ID','SEASON','TEAM_NAME','COMPETITION'], how='left')
team_defending = team_defending.merge(team_standard, on=['TEAM_FBREF_ID','SEASON','TEAM_NAME', 'COMPETITION'], how='left')

In [107]:
team_attacking = team_attacking.merge(df_competitions[['COMPETITION','COMPETITION_ACRONYM','SEASON']], 
                                      on=['COMPETITION','SEASON'], how='left')
team_defending = team_defending.merge(df_competitions[['COMPETITION','COMPETITION_ACRONYM','SEASON']], 
                                      on=['COMPETITION','SEASON'], how='left')

In [108]:
team_attacking['Goals/Game'] = team_attacking['GOALS_SCORED']/team_attacking['MATCHES_PLAYED']
team_attacking['Pass Completion (%)'] = team_attacking['PASS_COMPLETED']*100/team_attacking['PASS_ATTEMPTED']
team_attacking['Fouls Against/Game'] = team_attacking['FOULS_AGAINST']/team_attacking['MATCHES_PLAYED']
team_attacking['NPxG/Game'] = team_attacking['NPXG']/team_attacking['MATCHES_PLAYED']
team_attacking['Shots/Game'] = team_attacking['SHOTS']/team_attacking['MATCHES_PLAYED']
team_attacking['Shots On Target (%)'] = team_attacking['SHOTS_ON_TARGET']*100/team_attacking['SHOTS']
team_attacking['Take_ons Attempted/Game'] = team_attacking['TAKEONS_ATTEMPTED']/team_attacking['MATCHES_PLAYED']
team_attacking['Crosses into Pen Area'] = team_attacking['CROSSES_INTO_PA']

In [109]:
# team_defending['Clearances/Game'] = team_defending['CLEARANCES']/team_defending['MATCHES_PLAYED']
team_defending['Fouls Made/Game'] = team_defending['FOULS_MADE']/team_defending['MATCHES_PLAYED']
team_defending['Conceded/Game'] = team_defending['GOALS_CONCEDED']/team_defending['MATCHES_PLAYED']
team_defending['xG Against/Game'] = team_defending['XG_AGAINST']/team_defending['MATCHES_PLAYED']
# team_defending['Tackles Attempted/Game'] = team_defending['TACKLES']/team_defending['MATCHES_PLAYED']
team_defending['Tackles Won (%)'] = team_defending['TACKLES_WON']*100/team_defending['TACKLES']
# team_defending['Interceptions/Game'] = team_defending['INTERCEPTIONS']/team_defending['MATCHES_PLAYED']
# team_defending['Blocked Shots/Game'] = team_defending['BLOCKED_SHOTS']/team_defending['MATCHES_PLAYED']
team_defending['Possession Won'] = team_defending['POSS_WON']
team_defending['Opposition PPDA'] = team_defending['OPP_PPDA']
team_defending['Final 3rd Passes Against/Game'] = team_defending['FINAL_3RD_PASSES_AGAINST']/team_defending['MATCHES_PLAYED']
team_defending['Clean Sheets'] = team_defending['CLEAN_SHEETS']

In [110]:
team_attacking= team_attacking[['SEASON', 'TEAM_NAME','COMPETITION_ACRONYM' ,'Goals/Game', 'Pass Completion (%)', 'Fouls Against/Game', 'NPxG/Game', 
                                'Shots/Game', 'Shots On Target (%)', 'Take_ons Attempted/Game', 'Crosses into Pen Area']]


In [111]:
# team_defending= team_defending[['SEASON', 'TEAM_NAME','COMPETITION_ACRONYM' ,'Clearances/Game', 'Fouls Made/Game', 'Conceded/Game', 'xG Against/Game',
#                                 'Tackles Attempted/Game', 'Tackles Won (%)', 'Interceptions/Game', 'Blocked Shots/Game','Possession Won', 
#                                 'Opposition PPDA', 'Final 3rd Passes Against/Game', 'Clean Sheets']]

team_defending= team_defending[['SEASON', 'TEAM_NAME','COMPETITION_ACRONYM','MATCHES_PLAYED' ,'Fouls Made/Game', 'Conceded/Game', 
                                'xG Against/Game', 'Tackles Won (%)', 'Possession Won', 
                                'Opposition PPDA', 'Final 3rd Passes Against/Game', 'Clean Sheets']]

In [112]:
team_attacking_average = team_attacking.drop(columns=['TEAM_NAME']).groupby(['SEASON', 'COMPETITION_ACRONYM']).mean().reset_index()
team_attacking_average['TEAM_NAME'] = team_attacking_average['COMPETITION_ACRONYM'] +'_'+ team_attacking_average['SEASON'].astype(str) + '_Average'
team_attacking = pd.concat([team_attacking, team_attacking_average], ignore_index=True)

In [113]:
team_defending_average = team_defending.drop(columns=['TEAM_NAME']).groupby(['SEASON', 'COMPETITION_ACRONYM']).mean().reset_index()
team_defending_average['TEAM_NAME'] = team_defending_average['COMPETITION_ACRONYM'] +'_'+ team_defending_average['SEASON'].astype(str) + '_Average'
team_defending = pd.concat([team_defending, team_defending_average], ignore_index=True)

In [116]:
scaler = MinMaxScaler()
team_defending_scaled = (team_defending.drop(['TEAM_NAME','SEASON','COMPETITION_ACRONYM','MATCHES_PLAYED'], axis=1))
team_defending_scaled_1 = team_defending_scaled.drop(['Fouls Made/Game', 'Conceded/Game', 'xG Against/Game', 
                                                      'Final 3rd Passes Against/Game','Opposition PPDA'], axis=1)
team_defending_scaled_2 = team_defending_scaled[['Fouls Made/Game', 'Conceded/Game', 'xG Against/Game', 
                                                 'Final 3rd Passes Against/Game','Opposition PPDA']]

In [119]:
team_defending['MATCHES_PLAYED']

0     38.0
1     38.0
2     38.0
3     38.0
4     38.0
      ... 
58    20.0
59    20.0
60    38.0
61    38.0
62    20.3
Name: MATCHES_PLAYED, Length: 63, dtype: float64

In [123]:
team_defending_scaled_1['Possession Won'] = team_defending_scaled_1['Possession Won']/team_defending['MATCHES_PLAYED']
team_defending_scaled_1['Clean Sheets'] = team_defending_scaled_1['Clean Sheets']/team_defending['MATCHES_PLAYED']

In [124]:
team_defending_scaled_1

,Tackles Won (%),Possession Won,Clean Sheets
0,57.592593,87.289474,0.342105
1,60.252366,93.736842,0.289474
2,56.342669,102.526316,0.236842
3,60.329341,104.315789,0.289474
4,54.915254,107.815789,0.236842
...,...,...,...
58,66.123779,90.100000,0.250000
59,59.411765,88.450000,0.150000
60,57.581447,99.453947,0.278947
61,58.335138,97.877632,0.272368


In [44]:
scaler = MinMaxScaler()
team_defending_scaled = (team_defending.drop(['TEAM_NAME','SEASON','COMPETITION_ACRONYM'], axis=1))
team_defending_scaled_1 = team_defending_scaled.drop(['Fouls Made/Game', 'Conceded/Game', 'xG Against/Game', 
                                                      'Final 3rd Passes Against/Game','Opposition PPDA'], axis=1)
team_defending_scaled_2 = team_defending_scaled[['Fouls Made/Game', 'Conceded/Game', 'xG Against/Game', 
                                                 'Final 3rd Passes Against/Game','Opposition PPDA']]
team_defending_scaled_1['Possession Won'] = team_defending_scaled_1['Possession Won']/team_defending['MATCHES_PLAYED']
team_defending_scaled_1['Clean Sheets'] = team_defending_scaled_1['Clean Sheets']/team_defending['MATCHES_PLAYED']
team_defending_scaled_1 = pd.DataFrame(scaler.fit_transform(team_defending_scaled_1), columns=team_defending_scaled_1.columns)

In [45]:
scaler = MinMaxScaler()
team_defending_scaled_2 = pd.DataFrame(scaler.fit_transform(team_defending_scaled_2), columns=team_defending_scaled_2.columns)
team_defending_scaled_2 = pd.DataFrame(scaler.fit_transform(1-team_defending_scaled_2), columns=team_defending_scaled_2.columns)
team_defending_scaled_2 = pd.DataFrame(scaler.inverse_transform(team_defending_scaled_2), columns=team_defending_scaled_2.columns)
team_defending_scaled = pd.concat([team_defending_scaled_1, team_defending_scaled_2], axis=1)

In [46]:
team_defending_scaled = pd.concat([team_defending[['TEAM_NAME','SEASON','COMPETITION_ACRONYM']], team_defending_scaled], axis=1)

In [47]:
# scaler = MinMaxScaler()
# team_defending_scaled = (team_defending.drop(['TEAM_NAME','SEASON'], axis=1))
# team_defending_scaled_1 = team_defending_scaled.drop(['Fouls Made/Game', 'Conceded/Game', 'xG Against/Game'], axis=1)
# team_defending_scaled_2 = team_defending_scaled[['Fouls Made/Game', 'Conceded/Game', 'xG Against/Game']]
# team_defending_scaled_1 = pd.DataFrame(scaler.fit_transform(team_defending_scaled_1), columns=team_defending_scaled_1.columns)
# scaler = MinMaxScaler()
# team_defending_scaled_2 = pd.DataFrame(scaler.fit_transform(team_defending_scaled_2), columns=team_defending_scaled_2.columns)
# team_defending_scaled_2 = pd.DataFrame(scaler.fit_transform(1-team_defending_scaled_2), columns=team_defending_scaled_2.columns)
# team_defending_scaled_2 = pd.DataFrame(scaler.inverse_transform(team_defending_scaled_2), columns=team_defending_scaled_2.columns)
# team_defending_scaled = pd.concat([team_defending_scaled_1, team_defending_scaled_2], axis=1)
# team_defending_scaled = pd.concat([team_defending[['TEAM_NAME','SEASON']], team_defending_scaled], axis=1)

In [48]:
scaler = MinMaxScaler()
team_attacking_scaled = (team_attacking.drop(['TEAM_NAME','SEASON','COMPETITION_ACRONYM'], axis=1))
team_attacking_scaled = pd.DataFrame(scaler.fit_transform(team_attacking_scaled), columns=team_attacking_scaled.columns)
team_attacking_scaled = pd.concat([team_attacking[['TEAM_NAME','SEASON','COMPETITION_ACRONYM']], team_attacking_scaled], axis=1)

In [49]:
team_attacking_radar_1 = team_attacking_scaled.melt(id_vars=["SEASON", "TEAM_NAME","COMPETITION_ACRONYM"]).sort_values(
                        by=["COMPETITION_ACRONYM","SEASON","TEAM_NAME"]).rename(columns={'value':'norm_value'})
team_attacking_radar_2 = team_attacking.melt(id_vars=["SEASON", "TEAM_NAME","COMPETITION_ACRONYM"]).sort_values(
                        by=["COMPETITION_ACRONYM","SEASON","TEAM_NAME"])

In [50]:
team_defending_radar_1 = team_defending_scaled.melt(id_vars=["SEASON", "TEAM_NAME","COMPETITION_ACRONYM"]).sort_values(
                        by=["COMPETITION_ACRONYM","SEASON","TEAM_NAME"]).rename(columns={'value':'norm_value'})
team_defending_radar_2 = team_defending.melt(id_vars=["SEASON", "TEAM_NAME","COMPETITION_ACRONYM"]).sort_values(
                        by=["COMPETITION_ACRONYM","SEASON","TEAM_NAME"])

In [51]:
team_defending_radar = team_defending_radar_1.merge(team_defending_radar_2, on=['SEASON','TEAM_NAME','COMPETITION_ACRONYM','variable'], how='left')
team_attacking_radar = team_attacking_radar_1.merge(team_attacking_radar_2, on=['SEASON','TEAM_NAME','COMPETITION_ACRONYM','variable'], how='left')

In [52]:
team_standard_radar_1 = team_attacking_scaled[['TEAM_NAME','SEASON','COMPETITION_ACRONYM','Pass Completion (%)','Goals/Game',
                                               'NPxG/Game','Shots/Game','Shots On Target (%)']]
team_standard_radar_2 = team_attacking[['TEAM_NAME','SEASON','COMPETITION_ACRONYM','Pass Completion (%)','Goals/Game','NPxG/Game',
                                               'Shots/Game','Shots On Target (%)']]
team_standard_radar_3 = team_defending_scaled[['TEAM_NAME','SEASON','COMPETITION_ACRONYM','Tackles Won (%)', 'Conceded/Game', 
                                               'xG Against/Game']]
team_standard_radar_4 = team_defending[['TEAM_NAME','SEASON','COMPETITION_ACRONYM','Tackles Won (%)', 'Conceded/Game', 
                                        'xG Against/Game']]

In [53]:
team_standard_radar_5 = team_standard_radar_1.merge(team_standard_radar_3, on=['TEAM_NAME','SEASON','COMPETITION_ACRONYM'], how='left')
team_standard_radar_6 = team_standard_radar_2.merge(team_standard_radar_4, on=['TEAM_NAME','SEASON','COMPETITION_ACRONYM'], how='left')

In [54]:
team_standard_radar_5 = team_standard_radar_5.melt(id_vars=["SEASON", "TEAM_NAME", "COMPETITION_ACRONYM"]).sort_values(
    by=["COMPETITION_ACRONYM","SEASON","TEAM_NAME"]).rename(columns={'value':'norm_value'})

In [55]:
team_standard_radar_6 = team_standard_radar_6.melt(id_vars=["SEASON", "TEAM_NAME", "COMPETITION_ACRONYM"]).sort_values(
    by=["COMPETITION_ACRONYM","SEASON","TEAM_NAME"])

In [56]:
team_standard_radar = team_standard_radar_5.merge(team_standard_radar_6, on=['SEASON','TEAM_NAME','COMPETITION_ACRONYM','variable'], how='left')

In [57]:
team_defending_radar.rename(columns={'variable':'VARIABLE','norm_value':"NORM_VALUE",
                                     'value':'VALUE'}, inplace=True)

In [58]:
team_attacking_radar.rename(columns={'variable':'VARIABLE','norm_value':"NORM_VALUE",
                                     'value':'VALUE'}, inplace=True)

In [59]:
team_standard_radar.rename(columns={'variable':'VARIABLE','norm_value':"NORM_VALUE",
                                     'value':'VALUE'}, inplace=True)

Upload to Snowflake now

In [60]:
SNOWFLAKE_USER = 'kbharaj3'
SNOWFLAKE_PASSWORD = 'Snowfl@key0014'
SNOWFLAKE_ACCOUNT = 'qx25653.ca-central-1.aws'
SNOWFLAKE_WAREHOUSE = 'FOOTY_STORE'
SNOWFLAKE_DATABASE = 'GEGENSTATS'
SNOWFLAKE_SCHEMA = 'RADAR_CHARTS'

In [61]:
conn = snowflake.connector.connect(
    user=SNOWFLAKE_USER,
    password=SNOWFLAKE_PASSWORD,
    account=SNOWFLAKE_ACCOUNT,
    warehouse=SNOWFLAKE_WAREHOUSE,
    database=SNOWFLAKE_DATABASE,
    schema=SNOWFLAKE_SCHEMA
    )

In [62]:
cur = conn.cursor()
cur.execute(f"USE WAREHOUSE {SNOWFLAKE_WAREHOUSE}")

In [63]:
create_schema_sql = f"CREATE SCHEMA IF NOT EXISTS {SNOWFLAKE_SCHEMA}"
cur.execute(create_schema_sql)

In [219]:
create_table_query = """
CREATE TABLE IF NOT EXISTS STANDARD_RADAR (
    SEASON INT,
    TEAM_NAME VARCHAR(255),
    COMPETITION_ACRONYM VARCHAR(255),
    VARIABLE VARCHAR(255),
    NORM_VALUE FLOAT,
    VALUE FLOAT,
    PRIMARY KEY (SEASON, COMPETITION_ACRONYM, TEAM_NAME, VARIABLE)
);
"""

In [220]:
cur.execute(create_table_query)

In [64]:
def insert_to_snowflake_DB(primary_keys, table_name, dataframe, replace=False):
    if replace:
        cur.execute(f"TRUNCATE TABLE {table_name}")
        success, nchunks, nrows, _ = write_pandas(conn, dataframe, table_name)
    else:
        cur.execute(f"SELECT {', '.join(primary_keys)} FROM {table_name}")
        existing_ids = cur.fetchall()
        existing_ids = [id for id in existing_ids]
        dataframe.set_index(primary_keys, inplace=True)
        new_rows = dataframe[~dataframe.index.isin(existing_ids)]
        new_rows.reset_index(inplace=True)
        dataframe.reset_index(inplace=True)
        if not new_rows.empty:
            success, nchunks, nrows, _ = write_pandas(conn, new_rows, table_name)
            print(f"Inserted {nrows} new rows")
        else:
            print("No new rows to insert")

In [50]:
insert_to_snowflake_DB(['SEASON','COMPETITION_ACRONYM','TEAM_NAME','VARIABLE'], 'STANDARD_RADAR', team_standard_radar, 
                       replace=True)

In [223]:
create_table_query = """
CREATE TABLE IF NOT EXISTS ATTACKING_RADAR (
    SEASON INT,
    TEAM_NAME VARCHAR(255),
    COMPETITION_ACRONYM VARCHAR(255),
    VARIABLE VARCHAR(255),
    NORM_VALUE FLOAT,
    VALUE FLOAT,
    PRIMARY KEY (SEASON, COMPETITION_ACRONYM, TEAM_NAME, VARIABLE)
);
"""

In [51]:
cur.execute(create_table_query)

NameError: name 'create_table_query' is not defined

In [52]:
insert_to_snowflake_DB(['SEASON','COMPETITION_ACRONYM','TEAM_NAME','VARIABLE'], 'ATTACKING_RADAR', team_attacking_radar, 
                       replace=True)

In [226]:
create_table_query = """
CREATE TABLE IF NOT EXISTS DEFENDING_RADAR (
    SEASON INT,
    TEAM_NAME VARCHAR(255),
    COMPETITION_ACRONYM VARCHAR(255),
    VARIABLE VARCHAR(255),
    NORM_VALUE FLOAT,
    VALUE FLOAT,
    PRIMARY KEY (SEASON, TEAM_NAME, VARIABLE)
);
"""

In [227]:
cur.execute(create_table_query)

In [67]:
insert_to_snowflake_DB(['SEASON','COMPETITION_ACRONYM','TEAM_NAME', 'VARIABLE'], 'DEFENDING_RADAR', team_defending_radar, 
                       replace=True)

In [66]:
team_defending_radar.head(50)

,SEASON,TEAM_NAME,COMPETITION_ACRONYM,VARIABLE,NORM_VALUE,VALUE
0,2122,Arsenal,EPL,Tackles Won (%),0.329692,57.592593
1,2122,Arsenal,EPL,Possession Won,0.691905,3317.000000
2,2122,Arsenal,EPL,Clean Sheets,0.600000,13.000000
3,2122,Arsenal,EPL,Fouls Made/Game,0.637646,9.605263
4,2122,Arsenal,EPL,Conceded/Game,0.672131,1.263158
5,2122,Arsenal,EPL,xG Against/Game,0.655285,1.273421
6,2122,Arsenal,EPL,Final 3rd Passes Against/Game,0.598878,26.210526
7,2122,Arsenal,EPL,Opposition PPDA,0.583496,13.940000
8,2122,Aston Villa,EPL,Tackles Won (%),0.538674,60.252366
9,2122,Aston Villa,EPL,Possession Won,0.777390,3562.000000
